In [137]:
from brian2 import *

# Implementation
This is an attempt on implementing the learning algorithm described in [An autonomous learning mobile robot using biological reward modulate STDP](https://www.sciencedirect.com/science/article/pii/S0925231221009310)

# Setup the individual parts

## Input
We define our input using a PoissonGroup for now. Could change for some gym input if time.

In [138]:
start_scope()
num_input = 1
input_rate = 10*Hz

input = PoissonGroup(num_inputs, rates=input_rate)

## LIF:
We start by defining our LIF model  
Model: $\dfrac{dV}{dt} = \dfrac{(-V + V_{rest} + R_mI_{syn})}{\tau_m}$, where  
$V_{rest}$ is the resting potential, $R_m$ is the membrane resistance, $\tau_m$ is the decay time for the membrane potential and $I_{syn}$ is the synaptic injection current



In [139]:
# lif constants
v_rest = 0
R_m = 5*Gohm
tau_m = 10*ms
v_thresh = 15*mV

lif_model = """
dv/dt = (-v + v_rest + R_m * I_syn)/tau_m : volt
"""

neurons = NeuronGroup(2, lif_model, threshold="v>v_thresh", reset="v=v_rest", method="exact")

## STDP
We start off by defining our STDP synapse using the following equations:

## Synapse
**Synaptic injection current:**  
$\dfrac{dI_{syn}}{dt}  = \dfrac{(-I_{syn} + C_{syn}*\sum_i^Nw\delta(t-t_{pre}))}{\tau_{syn}}$, where  
$C_syn$ is the constant of injection current, $\tau_{syn}$ is the decay time, $w$ is the synaptic weight, $\delta$ is the dirac function and, $t_{pre}$ is the presynaptic spike time

## Learning algorithm
**Presynaptic spike trajectory:**  
$\dfrac{dT_{pre}}{dt} = -\dfrac{T_{pre}}{\tau_{pre}} + A_{+}\sum_{pre}\delta(t-t_{pre})$  

**Postsynaptic spike trajectory:**  
$\dfrac{dT_{post}}{dt} = -\dfrac{T_{post}}{\tau_{post}} + A_{-}\sum_{post}\delta(t-t_{pre})$ where,  

$T_{pre}$ is the presynaptic spike trajectories, $T_{post}$ is the postsynaptic spike trajectories, $\tau_{pre}$ is the presynatpic decay time, $\tau_{post}$ is the postsynatpic decay time, $A_{+}$ is the scaling factor for LTP, $A_{-}$ is the scaling factor for LTD, $\delta$ is the dirac function, $t_{pre}$ is the presynaptic spike time, and $t_{post}$ is the postsynaptic spike time.

**Eligbility trace to collect the pre and post-synaptic trajectory:**  
$\dfrac{det}{dt} = \dfrac{-T_{pre}\delta(t-t_{post}) - T_{post}\delta(t-t_{pre})}{\tau_{et}}$, where  
$et$ is the eiligbility trace and $\tau_{et}$ is the decay time.

**State division:**  
$\dfrac{dS_n}{dt} = -\dfrac{S_n}{\tau_s} + \sum_n\delta(t-t_{pre}^n)$, where  
$S_n$ represents the trajectory of the $n^{th}$ state space, $t_{pre}^n$ is the firing time of the presynaptic spike in the $n^{th}$ state space and $\tau_s$ is the decay time.

**Memory curve:**  
$\dfrac{dM_n}{dt} = \dfrac{S_net}{\tau_m}$, where  
$M_n$ represent the memory curve in the $n^{th}$ state space and $\tau_m$ is the decay time.

**Synaptic weight update:**  
$\dfrac{dw}{dt} = \dfrac{\gamma R(t) w \sum_MM_n[H(t-t_n^{start}) - H(t-t_n^{end})]}{\tau_w}$, where  
$\tau_m$ is the decay time of the synaptic weight, $\gamma$ is the learning rate, $R(t)$ is the reward function, $H(t)$ is the heaviside function and $t_n^{start}$, $t_n^{end}$ are the start and end moments of the $n^{th}$ memory curve.

**Reward function:**  
$\dfrac{dR(t)}{dt} = -\dfrac{R(t)}{\tau_R}+C_r\sum \pm \delta(t-t_R)$, where  
$R(t)$ is the reward, $t_r$ is the time for reward signal generation, $C_r$ is the reward constant and $\tau_R$ is the decay time.

In [140]:
# stdp constants
tau_pre = 20*ms # decay for pre-synaptic spike traces
tau_post = tau_pre # decay for post-synaptic spike traces
A_plus = 0.1 # LTD scaling factor
A_minus = -0.1 # LTP scaling factor
tau_et = 20*ms # time-decay for the eligbility trace
tau_sn = 20*ms # time-decay for the state division
tau_M = 20*ms # time-decay for the memory curve
tau_w = 20*ms # time-decay for the synaptic weight update
tau_r = 10*ms # time-decay for the reward function
gamma = 0.0025 # learning rate
w_0 = 1.1 # initial synaptic weight
C_R = 1 # reward constant
C_syn = 5 

# need to be stdp
# apre = apre
# apost = apost
# et = eligiblity trace
# sn = state division for the n:th state division
# mn = memory curve for the n:th state division
# r = reward function
# w = synaptic weight

# heavside can be defined as int(x) > 1
stdp_model = """
dI_syn/dt = -I_syn/tau_syn : 1 (event-driven)
dapre/dt = -apre/tau_pre : 1 (event-driven)
dapost/dt = -apost/tau_post : 1 (event-driven)
det/dt = et/tau_et : 1 (event-driven)
dsn/dt = -sn/tau_sn : 1 (event-driven)
dmn/dt = sn*et/tau_M : 1 (event-driven)
dr/dt = -r / tau_r : 1 (event-driven)
dw/dt = (gamma * rt * w)/tau_w : 1 (event-driven)
"""

stdp_pre = """
I_syn += (C_syn + w) / tau_syn
apre += A_plus
et += -apost
sn += 1
"""

stdp_post = """
apost += A_minus
et += -apre
"""

stdp = Synapses(input, neurons, model=stdp_model, on_pre=stdp_pre, on_post=stdp_post)

reward_model = """
flip : -1 : 1
r = C_R * flip : 1 (event-driven)
"""

#reward = Synapses()

# Setup the connections
We have our input, connected to our output neurons through 2 different STDP synapses. The STDP synapses are connected to do dopamine signalling.

In [141]:
stdp.connect(i=[0, 0], j=[0, 1])

# Run simulation

In [142]:
run(60*ms, report="text")

BrianObjectException: Error encountered with object named 'neurongroup_5'.
Object was created here (most recent call only, full details in debug log):
  File '/tmp/ipykernel_27919/3242513073.py', line 11, in <module>
    neurons = NeuronGroup(2, lif_model, threshold="v>v_thresh", reset="v=v_rest", method="exact")

An error occurred when preparing an object. (See above for original error message and traceback.)